In [24]:

def set_parameters(N1, p1, q1, d1):
    global N
    N = N1
    global p
    p = p1
    global q
    q = q1
    global d
    d = d1

# a class Zx of polynomials with integer coefficients and x as an unknown variable
Zx.<x> = ZZ[]

# ----------------------------------------------- AUXILIARY OPERATIONS ----------------------------------------------- 
def find_degree(coefs_list):
    """ 
    returns the degree of polynomial 
    """
    for i in range(len(coefs_list)-1, -1, -1):
        if coefs_list[i] != 0:
            return i

def invertmodprime(f,p):
    """
    Param1: a - polynom for inversion
    Param2: p - value for Z/Z_p
    
    Find inverse polynomial in Z/Z_p[X]/(X^N - 1)
    
    return: inverse polynom to 'a' in the factor ring
    """
    Zq.<z> = PolynomialRing(Integers(p))
    ZQphi.<Z> = Zq.quotient(z^N-1)
    a = f % p
    a = a.subs(x=z)


    k = 0
    b = 1*z^0
    c = 0*z^0
    f = a 
    g = z^N-1
    
    if a.gcd(g) != 1:
        raise Exception("inversion dosen't exist!")
        
    while True:
        while list(f)[0] == 0:
            f /= Z
            c *= Z
            k += 1
        
        if find_degree(list(f)) == 0:
            b = 1/list(f)[0] * b
            res = Z^(N-k) * b
            return Zx(res.lift())
        
        if find_degree(list(f)) < find_degree(list(g)):
            f, g = g, f
            b, c = c, b
        
        u = list(f)[0] * (1/list(g)[0])
        f -= u*g
        b -= u*c

        
def invertmodpowerof2(a, p):
    """
    Param1: a - polynom for inversion
    Param2: p - value for Z/Z_p
    
    Find inverse polynomial in Z/Z_p^r[X]/(X^N - 1)
    
    return: inverse polynom to 'a' in the factor ring
    """ 
    r = int(math.log(p, 2))
    p = 2
    
    q = p
    b = invertmodprime(a, p)

    while q < p^r:
        q = q^2
        b = b * (2 - a*b) % q % (x^N-1)
        
    b = b % p^r % (x^N - 1)
    return b
  

def balancedmod(f,q):
    ''' reduces every coefficient of a Zx polynomial f modulo q
        with additional balancing, so the result coefficients are integers in interval [-q/2, +q/2]
        more specifically: for an odd q [-(q-1)/2, +(q-1)/2], for an even q [-q/2, +q/2-1]. 
        returns Zx reduced polynomial'''

    g = list(((f[i] + q//2) % q) - q//2 for i in range(N))
    return Zx(g)

def convolution(f,g):
    ''' performs a multiplication operation specific for NTRU, which works like a traditional polynomial multiplication
        with additional reduction of the result by x^N-1 (x^n is replaced by 1, x^n-1 by x, x^n-2 by x^2, ...)
        returns Zx polynomial'''
    
    return (f * g) % (x^N-1)

# ----------------------------------------------- BASIC SETUP -----------------------------------------------
def validate_params():
    ''' checks params meet certain conditions: if q is considerably larger than p
        and if greatest common divider of p and q is 1 
        
        returns N, p, q '''
  
    if q > p and gcd(p,q) == 1:
        return True
    return False

def generate_polynomial(d1, d2):
    ''' generates a random polynomial with d nonzero coefficients
        returns Zx polynomial '''
    assert (d1 + d2) <= N       # asserting that there are less nonzero coefficients given than number of all coefficients
    
    result = [1]*d1 + [-1]*d2 + [0]*(N-d1-d2)  
    shuffle(result)
    
    return Zx(result)

# ----------------------------------------------- MAIN SETUP -----------------------------------------------
def generate_keys(polynomial_1 = None, polynomial_2= None):
    ''' generates a public and private key pair, based on provided parameters
        returns Zx public key and a secret key as a tuple of Zx f (private key) and Zx F_p'''

    # validate params
    if validate_params():
        #   some polynomials are not invertible and as f and g are calculated randomly,
        #   it may be necessary to skip some invalid examples
        while True:
            try:
                if polynomial_1 is None or polynomial_2 is None:   
                    # generate 2 random polynomials f and g with number of nonzero coefficients < given number
                    f = generate_polynomial(d+1, d)
                    g = generate_polynomial(d, d)
                else:
                    # it use your polynomials
                    f = polynomial_1
                    g = polynomial_2
                

                # formula: find f_q, where: f_q (*) f = 1 (mod q)
                # assuming q is a power of 2                 
                f_q = invertmodpowerof2(f,q)
                print("F_q = ", f_q)

                # formula: find f_p, where: f_p (*) f = 1 (mod p) 
                # assuming p is a prime number 
                f_p = invertmodprime(f,p)  
                print("F_p = ", f_p)
                break
        
            except:
                pass 
    
        #formula: public key = F_q ~ g (mod q)
        public_key = balancedmod(p * convolution(f_q,g),q)

        #secret key is a tuple containing a private key (f) and variable f_p needed for decryption
        secret_key = f,f_p

        return public_key,secret_key

    else:
        print("Provided params are not correct. q and p should be co-prime, q should be a power of 2 considerably larger than p and p should be prime.")

#---------------------------------- ENCRYPTION -----------------------------------------
def generate_message():
    ''' creates a polynomial from a random list of coefficients selected from a set {-1,0,1}  
        returns Zx polynomial'''
        
    #randrange(3) - 1 gives results from a set of {-1,0,1}, which is necessary for a proper decryption
    result = list(randrange(3) - 1 for j in range(N))
    return Zx(result)

def encrypt(message, public_key, r = None):
    ''' performs encryption of a given message using a provided public key
        returns Zx encrypted message'''

    # generate random polynomial with number of nonzero coefficients < N for adding extra noise  
    if r is None:
        r = generate_polynomial(d, d-1)
    
    # formula: encrypted_message = p * r ~ public_key + message (mod q)
    # while performing modulo operation, balance coefficients of encrypted_message 
    # for the integers in interval [-q/2, +q/2]
    return balancedmod(convolution(public_key,r) + message,q)


def decrypt(encrypted_message, secret_key):
    ''' performs decryption of a given ciphertext using an own private key
        
        returns Zx decrypted message'''
    
    # private key - f; additional variable stored for decryption - f_p     
    f,f_p = secret_key
    
    # formula: a = f ~ encrypted_message (mod q)
    # balance coefficients of a for the integers in interval [-q/2, +q/2]
    a = balancedmod(convolution(encrypted_message,f),q)
     
    # formula: F_p ~ a (mod p) with additional balancing as above
    return balancedmod(convolution(a,f_p),p)





In [25]:
# Wiki test
set_parameters(11, 3, 32, 2)
public_key, secret_key = generate_keys(-1+x+x^2-x^4+x^6+x^9-x^10, -1+x^2+x^3+x^5-x^8-x^10)
print("PUBLIC KEY = ", public_key.change_ring(Integers(q)))
message = -1+x^3-x^4-x^8+x^9+x^10
print("MESSAGE: " + str(message))
encrypted_message = encrypt(message, public_key)
print("ENCRYPTION: " + str(encrypted_message))

decrypted_message = decrypt(encrypted_message, secret_key)
print("DECRYPTION: " + str(decrypted_message))

if message == decrypted_message:
    print("SUCCESS")
else:
    print("FAIL")

F_q =  30*x^10 + 18*x^9 + 20*x^8 + 22*x^7 + 16*x^6 + 15*x^5 + 4*x^4 + 16*x^3 + 6*x^2 + 9*x + 5
F_p =  2*x^9 + x^8 + 2*x^7 + x^5 + 2*x^4 + 2*x^3 + 2*x + 1
PUBLIC KEY =  16*x^10 + 19*x^9 + 12*x^8 + 19*x^7 + 15*x^6 + 24*x^5 + 12*x^4 + 20*x^3 + 22*x^2 + 25*x + 8
MESSAGE: x^10 + x^9 - x^8 - x^4 + x^3 - 1
ENCRYPTION: -5*x^10 - 16*x^9 + 2*x^8 - 2*x^7 - 4*x^6 + x^5 - 10*x^4 - 16*x^3 - 11*x^2 + 7*x - 10
DECRYPTION: x^10 + x^9 - x^8 - x^4 + x^3 - 1
SUCCESS


In [27]:
set_parameters(821, 3, 4096, 60)
public_key, secret_key = generate_keys()
message = generate_message()


encrypted_message = encrypt(message, public_key)


decrypted_message = decrypt(encrypted_message, secret_key)


if message == decrypted_message:
    print("SUCCESS")
else:
    print("FAIL")

F_q =  1837*x^820 + 1449*x^819 + 1948*x^818 + 2914*x^817 + 3648*x^816 + 1592*x^815 + 871*x^814 + 1205*x^813 + 432*x^812 + 2905*x^811 + 665*x^810 + 2004*x^809 + 2884*x^808 + 3833*x^807 + 2307*x^806 + 2725*x^805 + 2371*x^804 + 1962*x^803 + 2557*x^802 + 2214*x^801 + 2070*x^800 + 2559*x^799 + 2535*x^798 + 2405*x^797 + 2714*x^796 + 386*x^795 + 857*x^794 + 3095*x^793 + 108*x^792 + 3623*x^791 + 584*x^790 + 2973*x^789 + 475*x^788 + 908*x^787 + 2827*x^786 + 1714*x^785 + 3861*x^784 + 3770*x^783 + 3319*x^782 + 3154*x^781 + 3514*x^780 + 3711*x^779 + 4027*x^778 + 2058*x^777 + 2139*x^776 + 1172*x^775 + 2632*x^774 + 2852*x^773 + 1994*x^772 + 3305*x^771 + 2393*x^770 + 3217*x^769 + 1061*x^768 + 488*x^767 + 2998*x^766 + 1120*x^765 + 664*x^764 + 843*x^763 + 2977*x^762 + 84*x^761 + 3424*x^760 + 2432*x^759 + 245*x^758 + 1008*x^757 + 235*x^756 + 216*x^755 + 2940*x^754 + 1622*x^753 + 1247*x^752 + 2855*x^751 + 513*x^750 + 2324*x^749 + 3613*x^748 + 474*x^747 + 3084*x^746 + 2943*x^745 + 1323*x^744 + 4026*x^743 

F_p =  x^820 + x^818 + 2*x^817 + x^813 + x^812 + 2*x^809 + x^807 + x^806 + 2*x^805 + 2*x^804 + x^800 + 2*x^799 + x^798 + x^797 + 2*x^796 + x^795 + x^794 + 2*x^793 + 2*x^792 + 2*x^791 + 2*x^790 + x^789 + x^788 + 2*x^787 + 2*x^785 + x^784 + 2*x^783 + x^782 + x^780 + x^779 + 2*x^778 + x^777 + 2*x^775 + x^774 + 2*x^773 + x^772 + x^769 + 2*x^768 + x^765 + 2*x^764 + x^761 + x^755 + x^754 + x^753 + x^751 + 2*x^750 + x^749 + 2*x^747 + 2*x^745 + x^744 + 2*x^742 + x^739 + 2*x^738 + x^735 + 2*x^733 + x^732 + x^731 + 2*x^730 + 2*x^729 + x^728 + 2*x^727 + x^725 + 2*x^723 + 2*x^722 + 2*x^721 + 2*x^720 + x^719 + 2*x^715 + 2*x^711 + x^709 + 2*x^707 + 2*x^706 + x^705 + x^704 + x^700 + x^697 + x^696 + x^695 + x^693 + 2*x^691 + x^690 + x^689 + x^687 + 2*x^685 + 2*x^683 + 2*x^680 + 2*x^679 + 2*x^678 + x^677 + x^676 + 2*x^675 + 2*x^674 + x^672 + 2*x^671 + x^670 + 2*x^667 + 2*x^665 + x^664 + 2*x^661 + 2*x^660 + 2*x^659 + x^656 + x^655 + 2*x^654 + 2*x^652 + x^649 + 2*x^648 + x^647 + x^645 + x^642 + 2*x^641 +